# Primo caso

- pip install scikit-learn
- pip install sentence-transformers
- pip install langdetect

In [ ]:
# Libreria che consente la rappresentazione vettoriale (embedded) di frasi anziché parole
from sentence_transformers import SentenceTransformer
# Libreria per calcolare la similarità coseno tra vettori (la similiarità coseno misura quanto due vettori sono simili)
from sklearn.metrics.pairwise import cosine_similarity
# Libreria per il rilevamento della lingua del testo
from langdetect import detect

# Carica modello multilingua potente che comprende l'italiano
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Lista statica di API con descrizioni ed endpoint
api_catalog = [
    {
        "descrizione": "Creazione nuovo finanziamento. Creazione per tipo e ndg",
        "endpoint": "api/finanziamento",
        "verbo": "POST",
    },
    {
        "descrizione": "Ricerca un finanziamento esistente. Ricerca per tipo e ndg",
        "endpoint": "api/finanziamento&tipo=mutuo&ndg=123456",
        "verbo": "GET",
    }
]

def get_api(testo_input, soglia_similarita=0.5):
    try:
        # Verifica che la lingua sia italiana
        if detect(testo_input) != "it":
            return "Per favore fornisci il testo in italiano."

        # Calcola embedding del testo utente
        embedding_input = model.encode([testo_input])

        migliori_match = {"endpoint": None, "score": 0.0}

        # Confronta con le descrizioni delle API
        for api in api_catalog:
            # Calcola embedding della descrizione dell'API
            embedding_descrizione = model.encode([api["descrizione"]])
            # Calcola la similarità coseno tra l'input e la descrizione dell'API
            sim = cosine_similarity(embedding_input, embedding_descrizione)[0][0]

            # Aggiorna il miglior match se la similarità è maggiore della soglia
            if sim > migliori_match["score"]:
                migliori_match = {"endpoint": api["endpoint"], "score": sim}

        # Controlla se il miglior match supera la soglia di similarità
        if migliori_match["score"] >= soglia_similarita:
            return f"{migliori_match['endpoint']}"
        else:
            return "La richiesta non trova corrispondenza con nessuna API. Riprova."

    except Exception as e:
        return f"Errore durante l'elaborazione: {str(e)}"


# Test primo caso

In [ ]:
esempi_richiesta = [
    "cercami i finanziamenti dell'ndg 123456",
    "voglio avviare un nuovo finanziamento",
    "mi cerchi un finanziamento",
    "ho bisogno di un mutuo per la casa",
    "vado a comprarmi una pizza",
    "mi crei un finanziamento con ndg 123456 e tipo mutuo",
]

for richiesta in esempi_richiesta:
    print(f"  Richiesta utente: {richiesta}")
    print(f"Endpoint associato: {get_api(richiesta)}\n")

# Secondo caso

- pip install spacy
- python -m spacy download it_core_news_md

# Importo librerie

In [42]:
# Libreria per l'elaborazione del linguaggio naturale
import spacy
# Libreria per operazioni numeriche e matriciali
import numpy as np
# Libreria per il modello di classificazione
from sklearn.linear_model import LogisticRegression
# Libreria che consente la rappresentazione vettoriale (embedded) di frasi anziché parole
from sentence_transformers import SentenceTransformer, util
# Libreria per calcolare la similarità coseno tra vettori (la similiarità coseno misura quanto due vettori sono simili)
from sklearn.metrics.pairwise import cosine_similarity
# Libreria per modelli di trasformatori (transformers)
from transformers import pipeline 
# Disabilita i warning di transformers
import transformers
transformers.logging.set_verbosity_error()
# Libreria per gestire tensori e modelli di deep learning
import torch 

# Addestramento machine learning 

In [8]:
X_train = [
    # ==== GET ====
    "cerca", "cercami", "fai una ricerca", "esegui una ricerca",
    "trova", "trovami", "rintraccia", "identifica",
    "recupera", "recuperami", "ottieni dati", "estrai informazioni",
    "mostra", "mostrami", "fammi vedere", "visualizza",
    "leggi", "leggimi", "accedi ai dati", "ottieni i dati",
    "visualizza", "visualizzami", "rendi visibile", "presenta",
    "vedi", "vedimi", "guarda i dati", "dammi una vista",
    "estrai", "estraimi", "scarica dati", "porta fuori dati",
    "accedi", "accedimi", "entra nei dati", "consulta",

    # ==== POST ====
    "crea", "creami", "genera nuovo", "costruisci un nuovo",
    "inserisci", "inseriscimi", "aggiungi", "carica nuovi dati",
    "richiedi", "richiedimi", "fai una richiesta", "manda una richiesta",
    "apri", "aprimi", "avvia una nuova pratica", "inizia procedura",
    "avvia", "avviami", "dai inizio", "comincia processo",
    "registra", "registrami", "salva nuovo", "archivia dati",
    "attiva", "attivami", "metti in funzione", "abilita",
    "compila", "compilami", "riempi i dati", "completa il modulo",

    # ==== PUT ====
    "aggiorna", "aggiornami", "fai un aggiornamento", "modifica con nuovi dati",
    "modifica", "modificami", "cambia i dati", "rivedi i valori",
    "correggi", "correggimi", "sistema dati", "risolvi errori",
    "rivedi", "rivedimi", "verifica e modifica", "ritocca",
    "sostituisci", "sostituiscimi", "cambia con altro", "scambia contenuto",
    "ricalcola", "ricalcolami", "rifai i conti", "esegui nuovo calcolo",

    # ==== DELETE ====
    "elimina", "eliminami", "cancella definitivamente", "rimuovi per sempre",
    "cancella", "cancellami", "butta via", "togli dai dati",
    "rimuovi", "rimuovimi", "escludi", "levami dai dati",
    "revoca", "revocami", "invalida", "annulla autorizzazione",
    "annulla", "annullami", "ferma l’operazione", "interrompi processo",
    "disattiva", "disattivami", "spegni", "rendi inattivo"
]

y_train = [
    # ==== GET ====
    *["GET"]*36,
    # ==== POST ====
    *["POST"]*32,
    # ==== PUT ====
    *["PUT"]*24,
    # ==== DELETE ====
    *["DELETE"]*24
]

Opzione 1

In [9]:
# Carica il modello italiano
nlp = spacy.load("it_core_news_md")

#metodo per trasformare x_train che contiene frasi in vettori numerici
def vectorizza(frasi):
    return np.array([nlp(frase).vector for frase in frasi])

# Vettorizza il training set
X_vect = vectorizza(X_train)

# Definisci il classificatore
clf = LogisticRegression()

# Allena il classificatore
clf.fit(X_vect, y_train)

# Metodo per estrarre verbo e complemento oggetto da una frase
def estrai_verbo_oggetto(frase: str, include_oggetto: bool = False):
    # Analizza la frase con spaCy
    doc = nlp(frase)
    risultati = []
    # Itera sui token della frase
    for token in doc:
        # Verbi
        if token.pos_ == "VERB":
            # Aggiungi il verbo alla lista dei risultati
            risultati.append(("VERBO", token.text))
        # Complemento oggetto (solo se richiesto dal flag)
        if include_oggetto and token.dep_ == "obj":
            # Aggiungi l'oggetto alla lista dei risultati
            risultati.append(("OGGETTO", token.text))
    return risultati

def classifica_http(testo_input: str, include_oggetto: bool = False):
    # Estraggo le caratteristiche del verbo (e oggetto se richiesto)
    caratteristiche_verbo = estrai_verbo_oggetto(testo_input, include_oggetto)
    if include_oggetto:
        # Estraggo sia la parola del verbo che dell'oggetto (escludendo etichetta e posizioni)
        verbo = [v[1] for v in caratteristiche_verbo if v[0] == "VERBO" or v[0] == "OGGETTO"]
    else:
        # Estraggo solo la parola del verbo (escludendo etichetta e posizioni)
        verbo = [v[1] for v in caratteristiche_verbo if v[0] == "VERBO"]
    # Vettorizzazione
    verbo_vect = vectorizza(verbo)
    # Predizione
    verbo_predetto = clf.predict(verbo_vect)[0]
    # Probabilità per tutte le classi
    probs = clf.predict_proba(verbo_vect)[0]   # array 1D tipo [0.05, 0.10, 0.82, 0.03]
    # Indice della classe predetta
    verbo_predetto_indice = clf.classes_.tolist().index(verbo_predetto)
    # Probabilità della classe predetta
    score_verbo_predetto = probs[verbo_predetto_indice]
    print("--- ML")
    print(f"       Verbo estratto: {verbo}")
    print(f"  Verbo http predetto: {verbo_predetto}")
    print(f"               Classi: {clf.classes_}")
    print(f"          Probabilità: {probs}")
    print(f"  Indice della classe: {verbo_predetto_indice}")
    print(f"Probabilità del verbo: {score_verbo_predetto}")
    #esempio
    #   Verbo http predetto       → 'POST'
    #   Classi                    → ['DELETE', 'GET', 'POST', 'PUT']
    #   Probabilità               → [0.05, 0.10, 0.82, 0.03]  # stesso ordine
    #   Indice della classe       → 2  (perché 'POST' è il 3° elemento)
    #   Probabilità del verbo     → verbo_probabilita[2] = 0.82
    return probs

Opzione 2

In [66]:
# Modello di embedding
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Calcolo gli embeddings dei tuoi esempi di training
X_emb = embedding_model.encode(X_train, convert_to_tensor=True)
""" 
# print(X_emb)
tensor([[ 0.012,  0.034, ..., -0.021],
        [ 0.020,  0.040, ..., -0.010],
        [ 0.015,  0.028, ..., -0.018],
        ...,
        [-0.012, 0.033, ..., 0.019]])
# print("Shape:", X_emb.shape)
Shape: torch.Size([116, 384]) 
"""

# Zero-shot fallback: 
# - zero-shot significa che si usa un modello pre-addestrato, che non necessita di addestramento sui tuoi esempi, per classificare i miei esempi
# - fallback perché uso prima un metodo principale e se la confidenza è bassa uso questo
# In poche parole, serve solo se vuoi gestire verbi/frasi molto fuori dai tuoi esempi

# Uso il modello BART di Facebook addestrato su MNLI (Multi-Genre Natural Language Inference)
# MNLI è un dataset di inferenza testuale, dove il modello impara a capire se una frase implica, contraddice o è neutrale rispetto a un'altra frase
zero_shot_model = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)

# Etichette HTTP
http_labels = ["GET", "POST", "PUT", "DELETE"]

def classifica_http_2(testo_input: str, include_oggetto: bool = False, top_k: int = 3, fallback_soglia: float = 0.65):
    print("--- NLP")
    """ 
    ESEMPIO
    "butta via i documenti"
    """
    
    # print("=== Step 1: Estrazione del verbo dall’input ===")
    caratteristiche_verbo = estrai_verbo_oggetto(testo_input, include_oggetto)
    # print("       Token estratti:", caratteristiche_verbo)
    if include_oggetto:
        # Estraggo sia la parola del verbo che dell'oggetto (escludendo etichetta e posizioni)
        verbo = [v[1] for v in caratteristiche_verbo if v[0] == "VERBO" or v[0] == "OGGETTO"]
    else:
        # Estraggo solo la parola del verbo (escludendo etichetta e posizioni)
        verbo = [v[1] for v in caratteristiche_verbo if v[0] == "VERBO"]
    # print("       Verbo estratto:", verbo)
    """ 
    Token estratti: [('VERBO','butta')]
    Verbo estratto: ['butta'] 
    """

    # print("\n=== Step 2: Calcolo dell’embedding del verbo estratto ===")
    verbo_emb = embedding_model.encode(verbo, convert_to_tensor=True)
    # print("       Verbo embedded:", verbo_emb.shape)
    """ 
    Verbo embedded: (1, 384)
    """

    # print("\n=== Step 3: Calcolo della similarità coseno con gli esempi di training ===")
    cos_scores = util.cos_sim(verbo_emb, X_emb)[0] 
    # print("    Similarità coseno:", cos_scores)
    """ 
    Similarità coseno: [0.45, 0.12, 0.08, 0.78, ...]
    """

    # print("\n=== Step 4: Selezione dei top_k esempi più simili ===")
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    preds = [y_train[i] for i in top_results] 
    verbo_predetto = preds[0] 
    confidenza = float(cos_scores[top_results[0]])
    # print("         Indici top_k:", top_results)
    # print("     Top_k similarità:", cos_scores[top_results])
    # print("         Classi top_k:", preds)
    """ 
    Indici top_k: [3, 6, 0]
    Valori top_k similarità: [0.78, 0.65, 0.45]
    Classi top_k: ['DELETE', 'GET', 'GET']
    """

    # print("\n=== Step 5: Predizione basata sulla classe più frequente ===")
    # conta quante volte compare ogni classe nei top_k
    counts = {label: preds.count(label) for label in set(preds)} 
    # perché prendere la classe con il conteggio massimo?
    # vantaggio:
    # - robustezza contro outlier: se uno dei top_k è un outlier
    # svantaggio:
    # - non considera la similarità: potrei avere 2 esempi "GET" con similarità 0.45 e 0.44, e 1 esempio "DELETE" con similarità 0.78
    verbo_predetto_2 = max(counts, key=counts.get)
    # print("     Conteggio classi:", counts)
    # print("   Classe più quotata:", verbo_predetto)
    """ 
    Conteggio classi: {'DELETE': 1, 'GET': 2}
    Classe più quotata: GET
    # print("Similarità max:", confidenza)
    Confidenza (similarità max): 0.78
    """

    # print("\n=== Step 6: Calcolo delle probabilità normalizzate per ogni classe ===")
    probs_dict = {}
    for label in http_labels:
        # prendo tutti i top_results che hanno questa label
        label_indices = [i for i in top_results if y_train[i] == label]
        if label_indices:
            # converto cos_scores in numpy solo se è un tensor
            cos_scores_np = cos_scores.cpu().detach().numpy() if isinstance(cos_scores, torch.Tensor) else cos_scores
            # media delle similarità per questa classe
            probs_dict[label] = float(np.mean(cos_scores_np[label_indices]))
        else:
            probs_dict[label] = 0.0
    # print(" Probabilità non norm:", probs_dict)
    # Normalizzo in modo che la somma sia 1
    total = sum(probs_dict.values())
    if total > 0:
        probs = [probs_dict[label]/total for label in http_labels]
    else:
        # fallback uniforme se total = 0
        probs = [1/len(http_labels)]*len(http_labels)
    # print("     Probabilità norm:", probs)
    """ 
    Probabilità non normalizzate: {'GET': 0.285, 'POST': 0.0, 'PUT': 0.0, 'DELETE': 0.78}
    Probabilità normalizzate: [0.27, 0.0, 0.0, 0.73]
    """

    # print("\n=== Step 7: Fallback zero-shot se la confidenza è bassa ===")
    if confidenza < fallback_soglia:
        zero_shot_res = zero_shot_model(testo_input, candidate_labels=http_labels)
        verbo_predetto_zero_shot = zero_shot_res['labels'][0]
        probs_zero_shot = [zero_shot_res['scores'][zero_shot_res['labels'].index(label)] for label in http_labels]
        # print("       Caso zero shot")
        # print("               Classi:", zero_shot_res['labels'])
        # print("          Probabilità:", zero_shot_res['scores'])
        # print("        Classe finale:", verbo_predetto_zero_shot)
        if zero_shot_res['scores'][0] > confidenza:
            print("       Caso zero shot")
            verbo_predetto = verbo_predetto_zero_shot
            probs = probs_zero_shot


    # print("\n--- Step 8: Risultato finale ---")
    print("               Classi:", http_labels)
    print("          Probabilità:", probs)
    print("      Classe predetta:", verbo_predetto)
    """ 
    Classe predetta: GET
    Probabilità di tutte le classi: [0.27, 0.0, 0.0, 0.73]
    """

    return probs


# Utils

In [11]:
# Metodo per analizzare il testo e suddividerlo in frasi
""" Il metodo riconosce più frasi separate da punteggiatura, invece, ad esempio, non riconosce invece frasi congiunte da "e" o "poi" """
def analizza_frasi(testo: str):
    # Analizza la frase con spaCy
    doc = nlp(testo)
    # Estrai le frasi dal testo
    frasi = [sent.text.strip() for sent in doc.sents] 
    print(f"       Frasi estratte: {frasi}")
    return len(frasi), frasi



# Api da individuare

In [12]:
# Lista statica di API con descrizioni ed endpoint
api_catalog = [
    {
        "descrizione": "Creazione nuovo finanziamento. Creazione per tipo e ndg",
        "endpoint": "api/finanziamento",
        "verbo_http": "POST"
    },
    {
        "descrizione": "Ricerca un finanziamento esistente. Ricerca per tipo e ndg",
        "endpoint": "api/finanziamento?tipo=mutuo&ndg=123456",
        "verbo_http": "GET"
    },
    {
        "descrizione": "Cancellazione finanziamento esistente. Cancellazione per ndg",
        "endpoint": "api/finanziamento?ndg=123456",
        "verbo_http": "DELETE"
    },
    {
        "descrizione": "Aggiornamento finanziamento esistente. Aggiornamento per ndg",
        "endpoint": "api/finanziamento?ndg=123456",
        "verbo_http": "PUT"
    }
]

# Algoritmo

In [72]:
# Carica modello multilingua potente che comprende l'italiano
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def get_api_2(testo_input, tipo_classificatore="ML", include_oggetto=False, ml_weight=0.5, nlp_weight=0.5, soglia_similarita=0.5):
    try:
        print(f"     Richiesta utente: {testo_input}")

        # Verifica che la richiesta contenga una sola frase
        num_frasi, lista_frasi = analizza_frasi(testo_input)
        if num_frasi > 1:
            return f"                       La richiesta contiene {num_frasi} frasi. Per favore invia una sola frase alla volta. Frasi rilevate: {lista_frasi}"
        
        # Recupera le probabilità per ogni verbo HTTP
        if tipo_classificatore == "ML":
            probs = classifica_http(testo_input, include_oggetto)
        else:
            probs = classifica_http_2(testo_input, include_oggetto)

        # Calcola embedding del testo utente
        embedding_input = model.encode([testo_input])

        migliore_match = {"endpoint": None, "verbo_http": None, "score": 0.0}
        migliore_match_composito = {"endpoint": None, "verbo_http": None, "score": 0.0}

        # Confronta con le descrizioni delle API
        for api in api_catalog:

            # Calcola embedding della descrizione dell'API
            embedding_api = model.encode([api["descrizione"]])

            # Calcola la similarità coseno tra l'input e la descrizione dell'API
            sim = cosine_similarity(embedding_input, embedding_api)[0][0]

            # Aggiorna il miglior match se la similarità è maggiore della soglia
            if sim > migliore_match["score"]:
                migliore_match = {"endpoint": api["endpoint"], "verbo_http": api["verbo_http"], "score": sim}

            # Combina i due score (similarità e probabilità verbo) con pesi alpha e beta
            if tipo_classificatore == "ML":
                verbo_indice = clf.classes_.tolist().index(api["verbo_http"])
            else:
                verbo_indice = http_labels.index(api["verbo_http"])
            score_verbo = probs[verbo_indice]
            sim_composito = ml_weight * score_verbo + nlp_weight * sim
            if sim_composito > migliore_match_composito["score"]:
                migliore_match_composito = {"endpoint": api["endpoint"], "verbo_http": api["verbo_http"], "score": sim_composito}
            

        print("--- NLP 2")
        print(f"    Similarità coseno: {migliore_match}")
        print(f"--- {tipo_classificatore} + NLP 2")

        # Controlla se il miglior match supera la soglia di similarità
        if migliore_match_composito["score"] >= soglia_similarita:
            return f"                       {migliore_match_composito}"
        else:
            return "                       La richiesta non trova corrispondenza con nessuna API. (score: " + str(migliore_match_composito["score"]) + ")"

    except Exception as e:
        return f"Errore durante l'elaborazione: {str(e)}"


# Test secondo caso

In [73]:
esempi_richiesta = [
    "cercami i finanziamenti dell'ndg 123456",
    "voglio avviare un nuovo finanziamento",
    "mi cerchi un finanziamento",
    "ho bisogno di un mutuo per la casa",
    "vado a comprarmi una pizza",
    "mi crei un finanziamento con ndg 123456 e tipo mutuo",
]

for richiesta in esempi_richiesta:
    print("----------------------------------------------------------------")
    print(get_api_2(richiesta, tipo_classificatore="ML", include_oggetto=False, ml_weight=0.1, nlp_weight=0.9, soglia_similarita=0.6)),
    print("----------------------------------------------------------------\n")

----------------------------------------------------------------
     Richiesta utente: cercami i finanziamenti dell'ndg 123456
       Frasi estratte: ["cercami i finanziamenti dell'ndg 123456"]
--- ML
       Verbo estratto: ['cercami']
  Verbo http predetto: GET
               Classi: ['DELETE' 'GET' 'POST' 'PUT']
          Probabilità: [9.66234521e-06 9.99464908e-01 5.23321003e-04 2.10856070e-06]
  Indice della classe: 1
Probabilità del verbo: 0.9994649080912499
--- NLP 2
    Similarità coseno: {'endpoint': 'api/finanziamento?tipo=mutuo&ndg=123456', 'verbo_http': 'GET', 'score': np.float32(0.71034634)}
--- ML + NLP 2
                       {'endpoint': 'api/finanziamento?tipo=mutuo&ndg=123456', 'verbo_http': 'GET', 'score': np.float64(0.739258162066144)}
----------------------------------------------------------------

----------------------------------------------------------------
     Richiesta utente: voglio avviare un nuovo finanziamento
       Frasi estratte: ['voglio avviare u

# TODO:
- Pensare a come gestire l'inserimento di più frasi da parte dell'utente in un'unica richiesta
- Migliorare la parte di ML, ad esempio: (Parzialmente fatto, TROVATA UN'ALTERNATIVA CON NLP)
  - cercare una libreria che data una parola ti fornisce tutti i sinonimi possibili (o una cosa del genere), potrebbe essere utile per costruire un dataset di input in modo automatico
  - ci sono modelli migliore del LogisticRegression? 
  - trovare una parametrizzazione efficiente per addestrare il modello scelto
- Capire come migliorare "live" l'algoritmo... renforce learning?
- AI può suuggerire le prossime azioni da fare (dettate dal workflow)?